In [2]:
#pip install vertexai faiss-cpu
#note scan must be built from source when I did it
import vertexai
import tensorflow as tf
from vertexai.language_models import TextEmbeddingInput
from vertexai.language_models import TextEmbeddingModel
import numpy as np
import os
from dotenv import load_dotenv
import tensorflow_recommenders as tfrs
import faiss
#vertex ai has many task types  "RETRIEVAL_QUERY", "RETRIEVAL_DOCUMENT",
# "SEMANTIC_SIMILARITY", # "CLASSIFICATION", and "CLUSTERING" for different purpose models


In [5]:
load_dotenv()
projectID = os.getenv('GOOGLE_VERTEX_PROJECT')
regionID = os.getenv('GOOGLE_REGION')
vertexai.init(project=projectID, location=regionID)

In [6]:
#textToEmbed = "This is the text embedded by the google model"
embeddingModel = TextEmbeddingModel.from_pretrained("textembedding-gecko@003")

#as a layer
#def Embed(text, task):
#    def embedText(text):
#        textInput = TextEmbeddingInput(task_type=task, text=text)
#        return np.array(embeddingModel.get_embeddings([textInput])[0].values)
#    output = tf.py_function(func=embedText, inp=[text], Tout=tf.float32)
#    output.set_shape((768,))
#    return output

def Embed(text, task):
    textInput = TextEmbeddingInput(task_type=task, text=text)
    return embeddingModel.get_embeddings([textInput])[0].values
#LLM_EMBEDDINGS = trainDataset.map(lambda x, y: Embed(x), y)

In [7]:
import sys #when i was trying to configure bazel for scann
textToSearch = [
    "John really likes his pizza",
    "The sky is blue",
    "John's pizza is a circle",
    "John has severe issues",
    "John needs to escape the horrible situtation he is in",
    "John must run until he can escape",
    "Spiralling spiralling spiralling",
    "Oh look a squirell",
    "John dies a brutal death none could have foreseen",
    "Water is important to drink make sure you get enough"
]
query = "what shape is the pizza John bought?"
print(sys.executable)

c:\Users\wyatt\AI\AIModels\LLMRandom\A-Collection-of-LLM-code\.conda\python.exe


In [13]:
# get embedded docs and query. Convert docs into a dataset
embedQuery = tf.constant((Embed(query, "RETRIEVAL_QUERY")))
embedSearch = np.array([Embed(value, "RETRIEVAL_DOCUMENT") for value in textToSearch])
#embedSearch = tf.data.Dataset.from_tensor_slices(embedSearch).enumerate().batch(1)

In [31]:
index = faiss.IndexHNSWFlat(768, 3) #768 dimensions because embedding dim #3 layers
index.add(embedSearch)
indexes = index.search(np.expand_dims(embedQuery, axis=0), k=2)
print(indexes)
sentences = indexes[1]
print(sentences)
for i in sentences[0]:
    print(textToSearch[i])

(array([[0.51821995, 0.5773435 ]], dtype=float32), array([[2, 0]], dtype=int64))
[[2 0]]
John's pizza is a circle
John really likes his pizza


In [ ]:
scan = tfrs.layers.factorized_top_k.ScaNN(distance_measure='dot product', num_leaves=5, num_leaves_to_search=2)
scan.index_from_dataset(embedSearch)
scan(embedQuery, k = 2)

ImportError: The scann library is not present. Please install it using `pip install scann` to use the ScaNN layer.